In [1]:
cd "/Users/sylviathsu/Documents/COS243/LocalAILibrarian"

/Users/sylviathsu/Documents/COS243/LocalAILibrarian


In [2]:
!pip install -q \
  llama-index \
  EbookLib \
  html2text \
  gradio \
  llama-index-embeddings-huggingface \
  llama-index-llms-ollama

In [3]:
import os
import logging
import gradio as gr
from llama_index.core import (
    SimpleDirectoryReader, 
    VectorStoreIndex, 
    StorageContext, 
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter
from pathlib import Path

/opt/anaconda3/envs/gradio/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def process_documents(doc_folder: str):
    """Process documents from the specified folder"""
    try:
        reader = SimpleDirectoryReader(
            input_dir=doc_folder,
            recursive=True,
            filename_as_id=True,
            required_exts=[".txt", ".epub", ".pdf"]
        )
        return reader.load_data()
    except Exception as e:
        logger.error(f"Document processing error: {e}")
        raise

def generate_embeddings(documents):
    """Generate embeddings and create vector store index"""
    try:
        # Configure embedding model
        embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
        
        # Configure text splitting
        splitter = SentenceSplitter(
            chunk_size=512,
            chunk_overlap=50,
            paragraph_separator="\n\n",
            include_metadata=True
        )
        
        # Create and save index
        vector_index = VectorStoreIndex.from_documents(
            documents,
            embed_model=embed_model,
            transformations=[splitter],
            show_progress=True
        )
        
        vector_index.storage_context.persist(persist_dir="storage")
        return vector_index
    except Exception as e:
        logger.error(f"Embedding generation error: {e}")
        raise

def configure_query_engine():
    """Configure the query engine with optimized parameters"""
    try:
        # Configure LLM with increased timeout
        llm = Ollama(
            model="phi3.5:3.8b-mini-instruct-q4_K_M",
            request_timeout=120.0,
            temperature=0.1
        )
        
        storage_context = StorageContext.from_defaults(persist_dir="storage")
        index = load_index_from_storage(storage_context)
        
        query_engine = index.as_query_engine(
            llm=llm,
            similarity_top_k=3,
            response_mode="compact",
            streaming=False
        )
        
        return query_engine
    except Exception as e:
        logger.error(f"Query engine configuration error: {e}")
        raise

def create_gradio_interface():
    """Create and configure the Gradio interface"""
    # Initialize query engine at the start
    global_query_engine = configure_query_engine() if Path("storage").exists() else None
    
    def handle_file_upload(files, progress=gr.Progress()):
        """Handle file upload and index update"""
        try:
            nonlocal global_query_engine
            
            # Create library directory if it doesn't exist
            Path("library").mkdir(exist_ok=True)
            
            # Copy uploaded files to library directory
            progress(0, desc="Copying files...")
            for file in files:
                shutil.copy2(file.name, "library")
            
            # Process documents and update index
            progress(0.3, desc="Processing documents...")
            documents = process_documents("library")
            
            progress(0.6, desc="Generating embeddings...")
            generate_embeddings(documents)
            
            progress(0.9, desc="Configuring query engine...")
            global_query_engine = configure_query_engine()
            
            progress(1.0, desc="Done!")
            return "Files uploaded and indexed successfully!"
        except Exception as e:
            logger.error(f"Upload error: {e}")
            return f"Error processing files: {str(e)}"

    def query_docs(query: str, history: List[Tuple[str, str]]) -> Tuple[str, List]:
        nonlocal global_query_engine
        
        if global_query_engine is None:
            error_msg = "Please upload documents first or wait for indexing to complete."
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
        
        try:
            response = global_query_engine.query(query)
            
            # Extract source information
            sources = []
            if hasattr(response, 'source_nodes'):
                sources = [
                    f"- {node.node.metadata.get('file_name', 'Unknown')}"
                    for node in response.source_nodes
                ]
            
            source_text = "\nSources:\n" + "\n".join(sources) if sources else ""
            full_response = str(response) + source_text
            
            history.append((query, full_response))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history
            
        except Exception as e:
            logger.error(f"Query processing error: {e}")
            error_msg = f"Error processing query: {str(e)}"
            history.append((query, error_msg))
            return "\n".join([f"Q: {q}\nA: {a}" for q, a in history]), history

    with gr.Blocks(title="Local AI Librarian") as app:
        gr.Markdown("# Local AI Librarian")
        gr.Markdown("Upload documents and search through them using natural language queries.")
        
        # Document upload section
        with gr.Row():
            with gr.Column():
                file_output = gr.Textbox(label="Upload Status")
                upload_button = gr.File(
                    file_count="multiple",
                    label="Upload Documents",
                    file_types=[".txt", ".pdf", ".epub"]
                )
        
        # Query section
        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Enter your query",
                    placeholder="What would you like to know about your documents?"
                )
                examples = gr.Examples(
                    examples=[
                        "What are the main themes in the documents?",
                        "Summarize the key points about...",
                        "Find relevant passages about..."
                    ],
                    inputs=query_input
                )
                
        with gr.Row():
            submit_btn = gr.Button("Search")
            clear_btn = gr.Button("Clear History")
            
        chat_history = gr.State([])
        output = gr.Textbox(
            label="Results",
            lines=20,
            autoscroll=False
        )
        
        # Set up event handlers
        upload_button.upload(
            handle_file_upload,
            inputs=[upload_button],
            outputs=[file_output]
        )
        
        submit_btn.click(
            query_docs,
            inputs=[query_input, chat_history],
            outputs=[output, chat_history]
        )
        
        clear_btn.click(
            lambda: ([], []),
            outputs=[output, chat_history]
        )
        
    return app

if __name__ == "__main__":
    try:
        logger.info("Starting application...")
        
        # Launch Gradio interface
        logger.info("Launching Gradio interface...")
        app = create_gradio_interface()
        app.launch(share=False)
        
    except Exception as e:
        logger.error(f"Application error: {e}")
        raise

INFO:__main__:Starting application...
INFO:__main__:Launching Gradio interface...
INFO:llama_index.core.indices.loading:Loading all indices.
INFO:httpx:HTTP Request: GET http://127.0.0.1:7866/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7866/ "HTTP/1.1 200 OK"


Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
